In [ ]:
#Face extraction using YOLO
from ultralytics import YOLO
import cv2
import os
import torch
from super_image import EdsrModel
from PIL import Image
import numpy as np

# Load the YOLO model
model = YOLO('yolov8n-face.pt')

# Input image path
input_image_path = 'classroom.jpg'
output_folder = 'extracted_faces/'

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read the input image
image = cv2.imread(input_image_path)
annotated_image = image.copy()

# Get the image dimensions (height, width)
img_height, img_width, _ = image.shape

# Padding factor (increase face box size by 10% on each side)
padding_ratio = 0.1

# Load the super-resolution model (EDSR model from super-image)
sr_model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=4)
sr_model.eval()  # Set model to evaluation mode

# Perform face detection using YOLO
results = model(image)

# Loop through detections
for i, (xmin, ymin, xmax, ymax, confidence, cls) in enumerate(results[0].boxes.data.tolist()):
    # Convert coordinates to integers
    xmin, ymin, xmax, ymax = map(int, [xmin, ymin, xmax, ymax])

    # Calculate padding size (10% of the bounding box size)
    box_width, box_height = xmax - xmin, ymax - ymin
    pad_x = int(box_width * padding_ratio)
    pad_y = int(box_height * padding_ratio)

    # Expand the bounding box, ensuring it stays within image boundaries
    xmin = max(0, xmin - pad_x)
    ymin = max(0, ymin - pad_y)
    xmax = min(img_width, xmax + pad_x)
    ymax = min(img_height, ymax + pad_y)

    # Extract the (expanded) face region
    face = image[ymin:ymax, xmin:xmax]
    # Convert the face to a PIL image
    face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
    face_pil= face_pil.resize((64,64), Image.Resampling.LANCZOS)
    # Convert PIL image to tensor
    face_tensor = torch.from_numpy(np.array(face_pil)).float().permute(2, 0, 1) / 255.0  # Normalize to [0, 1] range
    face_tensor = face_tensor.unsqueeze(0)  # Add batch dimension

    # Apply super-resolution using the EDSR model
    with torch.no_grad():  # Disable gradient calculation
        face_sr_tensor = sr_model(face_tensor)

    # Convert tensor back to PIL image
    face_sr = face_sr_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    face_sr = np.uint8(face_sr * 255)  # Convert to [0, 255] range

    # Convert back to OpenCV format and save the super-resolved face image
    output_path = os.path.join(output_folder, f'face_{i + 1}_sr.jpg')
    cv2.imwrite(output_path, cv2.cvtColor(face_sr, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 100])

    print(f'Saved: {output_path}')

    # Draw expanded bounding boxes and labels on annotated image
    cv2.rectangle(annotated_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(
        annotated_image, f'{i + 1}', (xmin, ymin - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA
    )

# Save and open the annotated image
cv2.imwrite('annotated_image.jpg', annotated_image)
os.startfile('annotated_image.jpg')  # Open with default viewer

print("Face extraction, super-resolution, and annotation completed.")


https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt

0: 320x640 56 faces, 48.6ms
Speed: 2.0ms preprocess, 48.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
Saved: extracted_faces/face_1_sr.jpg
Saved: extracted_faces/face_2_sr.jpg
Saved: extracted_faces/face_3_sr.jpg
Saved: extracted_faces/face_4_sr.jpg
Saved: extracted_faces/face_5_sr.jpg
Saved: extracted_faces/face_6_sr.jpg
Saved: extracted_faces/face_7_sr.jpg
Saved: extracted_faces/face_8_sr.jpg
Saved: extracted_faces/face_9_sr.jpg
Saved: extracted_faces/face_10_sr.jpg
Saved: extracted_faces/face_11_sr.jpg
Saved: extracted_faces/face_12_sr.jpg
Saved: extracted_faces/face_13_sr.jpg
Saved: extracted_faces/face_14_sr.jpg
Saved: extracted_faces/face_15_sr.jpg
Saved: extracted_faces/face_16_sr.jpg
Saved: extracted_faces/face_17_sr.jpg
Saved: extracted_faces/face_18_sr.jpg
Saved: extracted_faces/face_19_sr.jpg
Saved: extracted_faces/face_20_sr.jpg
Saved: extracted_faces/face_21_sr.jpg
Saved: e

In [ ]:
#Face extraction from references
import os
import cv2
from ultralytics import YOLO

# Define the paths
input_folder = 'reference_faces'  # Path to your folder with images
output_folder = 'ref_extracted/'  # Path to the folder where you want to save faces

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLO model (you can specify a specific model like 'yolov5s.pt' for the small model)
model = YOLO('yolov8n-face.pt')  # Load the YOLOv8 model

# Process each image in the input folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    
    # Read the image
    img = cv2.imread(image_path)
    
    # Perform face detection
    results = model(img)

    # Extract faces from detected results
    for i, (xyxy, conf, cls) in enumerate(zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls)):
        # Check if the detected class is a person (you may need to check the class ID for your specific model)
        if int(cls) == 0:  # 0 corresponds to person in YOLO models
            # Extract the face
            x1, y1, x2, y2 = map(int, xyxy)
            face = img[y1:y2, x1:x2]
            
            # Define the output path for the face image
            face_image_name = f"face_{image_name.split('.')[0]}_{i}.jpg"
            face_image_path = os.path.join(output_folder, face_image_name)
            
            # Save the face image
            cv2.imwrite(face_image_path, face)

print("Face extraction completed. Check the output folder.")



0: 640x640 1 face, 128.0ms
Speed: 5.0ms preprocess, 128.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 90.2ms
Speed: 4.0ms preprocess, 90.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 94.4ms
Speed: 5.0ms preprocess, 94.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 94.2ms
Speed: 5.0ms preprocess, 94.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 172.0ms
Speed: 6.0ms preprocess, 172.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 96.0ms
Speed: 5.0ms preprocess, 96.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 95.0ms
Speed: 5.0ms preprocess, 95.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 112.0ms
Speed: 6.0ms preprocess, 112.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0

In [ ]:
#References augmentation
import os
from PIL import Image
from torchvision import transforms

# Input and output folder
input_folder = 'ref_extracted'
output_folder = 'augmented_images'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define the augmentations
augmentation_pipeline = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomVerticalFlip(),    # Random vertical flip
    transforms.RandomRotation(30),      # Random rotation between -30 and 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),  # Random affine transformation
    transforms.RandomPerspective(distortion_scale=0.5, p=1.0, interpolation=3),  # Random perspective
])

# Loop through all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Process only image files
        image_path = os.path.join(input_folder, filename)

        # Open the image
        image = Image.open(image_path)

        # Create a folder for the current image based on its filename (without extension)
        person_folder = os.path.join(output_folder, os.path.splitext(filename)[0])
        os.makedirs(person_folder, exist_ok=True)

        # Generate 15 augmentations for each image
        for i in range(15):
            # Apply augmentations
            augmented_image = augmentation_pipeline(image)

            # Save the augmented image with a unique name
            output_path = os.path.join(person_folder, f'{os.path.splitext(filename)[0]}_aug_{i+1}.jpg')
            augmented_image.save(output_path)
            print(f'Saved: {output_path}')

print("Augmentation completed.")


Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_1.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_2.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_3.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_4.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_5.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_6.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_7.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_8.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_9.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_10.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_11.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_12.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_13.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_14.jpg
Saved: augmented_images\face_22bce002_0\face_22bce002_0_aug_15.jpg
Save

In [ ]:
#Training custom Siamese model
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import backend as K
from itertools import combinations
import random

# Triplet Loss function
def triplet_loss(anchor, positive, negative, alpha=0.5):
    # Normalize embeddings to unit vectors
    anchor = K.l2_normalize(anchor, axis=-1)
    positive = K.l2_normalize(positive, axis=-1)
    negative = K.l2_normalize(negative, axis=-1)
    
    # Calculate cosine similarity (dot product after normalization)
    pos_similarity = K.sum(anchor * positive, axis=-1)
    neg_similarity = K.sum(anchor * negative, axis=-1)
    
    # Define the triplet loss
    loss = K.maximum(neg_similarity - pos_similarity + alpha, 0.0)
    return K.mean(loss)

# Create a FaceNet model (Using MobileNetV2 as the base model for simplicity)
def create_facenet_model(input_shape=(224, 224, 3)):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)  # Embedding layer
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Path to the dataset
dataset_path = 'augmented_images'

# Prepare data (triplets of anchor, positive, and negative images)
def prepare_triplets(dataset_path):
    triplets = []
    class_names = os.listdir(dataset_path)
    
    for person in class_names:
        person_folder = os.path.join(dataset_path, person)
        images = os.listdir(person_folder)
        
        # We need at least 2 images to form anchor-positive pairs
        if len(images) < 2:
            continue
        
        # Create all possible anchor-positive pairs within the same class
        positive_pairs = list(combinations(images, 2))
        
        # Get all images for the other classes for negative sampling
        other_classes = [p for p in class_names if p != person]
        
        for anchor_image_name, positive_image_name in positive_pairs:
            anchor_image = os.path.join(person_folder, anchor_image_name)
            positive_image = os.path.join(person_folder, positive_image_name)
            
            # For each positive pair, pair with each image from other classes as negative samples
            for negative_person in other_classes:
                negative_folder = os.path.join(dataset_path, negative_person)
                negative_images = os.listdir(negative_folder)
                
                for negative_image_name in negative_images:
                    negative_image = os.path.join(negative_folder, negative_image_name)
                    triplets.append((anchor_image, positive_image, negative_image))
    
    return triplets
# Load image and preprocess
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize as the model expects
    return img_array

# Prepare triplets
triplets = prepare_triplets(dataset_path)

# Create embedding model
embedding_model = create_facenet_model(input_shape=(224, 224, 3))

# Optimizer
optimizer = optimizers.Adam(learning_rate=0.0001)

# Training loop
epochs = 10
batch_size = 32

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    epoch_loss = []
    
    # Shuffle the triplets
    random.shuffle(triplets)
    
    for i in range(0, len(triplets), batch_size):
        batch_triplets = triplets[i:i+batch_size]
        
        batch_anchor_embeddings = []
        batch_positive_embeddings = []
        batch_negative_embeddings = []
        
        with tf.GradientTape() as tape:
            for triplet in batch_triplets:
                anchor_image = load_and_preprocess_image(triplet[0])
                positive_image = load_and_preprocess_image(triplet[1])
                negative_image = load_and_preprocess_image(triplet[2])
                
                # Get embeddings
                anchor_embedding = embedding_model(anchor_image)
                positive_embedding = embedding_model(positive_image)
                negative_embedding = embedding_model(negative_image)
                
                batch_anchor_embeddings.append(anchor_embedding)
                batch_positive_embeddings.append(positive_embedding)
                batch_negative_embeddings.append(negative_embedding)
            
            # Stack embeddings for batch
            anchor_embeddings = tf.stack(batch_anchor_embeddings)
            positive_embeddings = tf.stack(batch_positive_embeddings)
            negative_embeddings = tf.stack(batch_negative_embeddings)
            
            # Calculate loss
            loss = triplet_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
            epoch_loss.append(loss.numpy())
        
        # Compute gradients and apply backpropagation
        gradients = tape.gradient(loss, embedding_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, embedding_model.trainable_variables))
    
    print(f"Loss: {np.mean(epoch_loss)}")

# Save the embedding model
get_custom_objects()['triplet_loss'] = triplet_loss
embedding_model.save('facenet_embedding_model.h5')


Epoch 1/10


KeyboardInterrupt: 

In [ ]:
#Inference from custom siamese model
import os
import numpy as np
from tensorflow.keras.models import load_model
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import cv2
import numpy as np
from numpy.linalg import norm

# Function to preprocess the images and compute embeddings
def preprocess_and_get_embedding(image_path, model):
    # Load and preprocess image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (224, 224))  # Resize to 160x160 as required by FaceNet

    # Normalize the image
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    # Predict embedding
    embedding = model.predict(image)
    return embedding

# Load the trained FaceNet model
model = load_model('facenet_embedding_model.h5')  # Replace with your actual model path

# Folder containing the extracted faces to predict
extracted_faces_folder = 'extracted_faces'
# Folder containing the reference (known) faces
ref_extracted_folder = 'ref_extracted'

# Store embeddings of reference (known) faces
ref_embeddings = {}
for filename in os.listdir(ref_extracted_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(ref_extracted_folder, filename)
        # Get the embedding for the current reference face
        embedding = preprocess_and_get_embedding(img_path, model)
        # Store the embedding for each person using the filename as the key
        person_name = os.path.splitext(filename)[0]  # Use the filename (without extension) as the person's name
        ref_embeddings[person_name] = embedding

output_file = 'predictions.txt'

with open(output_file, 'w') as file:
    for filename in os.listdir(extracted_faces_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            face_path = os.path.join(extracted_faces_folder, filename)
        
        # Get the embedding for the current extracted face
            extracted_face_embedding = preprocess_and_get_embedding(face_path, model)
        # Compare with reference embeddings using cosine similarity
            similarities = {}
            for person, ref_embedding in ref_embeddings.items():
            # Calculate cosine similarity
                similarity = similarity = cosine_similarity(extracted_face_embedding, ref_embedding.reshape(1, -1))[0][0]
                similarities[person] = similarity
            predicted_identity = max(similarities, key=similarities.get)
            similarity_score = similarities[predicted_identity]
            result = f"Predicted identity for {filename}: {predicted_identity} with similarity {similarity_score:.2f}\n"
            file.write(result)  # Save to file
            print(result)  # Optionally print to the console as well

print(f"Predictions saved to {output_file}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━

In [ ]:
#Predictions using FaceNet
import numpy as np
import cv2
import os
from deepface import DeepFace

# Constants
IMG_SIZE = (160, 160)  # Input size for FaceNet
THRESHOLD = 0.6  # Cosine similarity threshold for matching (higher value for stricter matching)

# Helper function to load and preprocess an image
def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMG_SIZE)  # Resize to (160, 160)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    return img

# Directories for extracted and reference faces
extracted_faces_dir = 'extracted_faces/'
reference_faces_dir = 'ref_extracted/'

matches = {}
# Prepare a text file to store matches
with open('matches.txt', 'w') as f:
    f.write("Extracted Face -> Reference Face (Similarity Score)\n")
    f.write("=" * 50 + "\n")

    # Load embeddings for reference faces using the same model for consistency
    reference_embeddings = {}
    for ref_file in os.listdir(reference_faces_dir):
        ref_path = os.path.join(reference_faces_dir, ref_file)
        ref_img = load_image(ref_path)

        # Use DeepFace to extract embeddings
        ref_embedding = DeepFace.represent(ref_img, model_name='Facenet', enforce_detection=False)[0]['embedding']
        reference_embeddings[ref_file] = ref_embedding

    # Compare each extracted face with all reference faces
    for extracted_file in os.listdir(extracted_faces_dir):
        extracted_path = os.path.join(extracted_faces_dir, extracted_file)
        extracted_img = load_image(extracted_path)

        # Use DeepFace to extract embeddings for the extracted face
        extracted_embedding = DeepFace.represent(extracted_img, model_name='Facenet', enforce_detection=False)[0]['embedding']

        # Initialize variables to keep track of the best match
        best_match = None
        best_score = -1  # Initialize with the worst possible score

        # Find the best match based on cosine similarity
        for ref_file, ref_embedding in reference_embeddings.items():
            # Calculate cosine similarity
            score = np.dot(extracted_embedding, ref_embedding) / (np.linalg.norm(extracted_embedding) * np.linalg.norm(ref_embedding))

            # Update the best match if the current score is better
            if score > best_score:
                best_score = score
                best_match = ref_file

        # Check if the best match exceeds the threshold
        if best_score > THRESHOLD:
            match_info = f"{extracted_file} -> {best_match} (Score: {best_score:.2f})\n"
            print(match_info.strip())
            f.write(match_info)

            # Store match details in dictionary
            if best_match not in matches:
                matches[best_match] = [(extracted_file, best_score)]
            else:
                matches[best_match].append((extracted_file, best_score))
        else:
            print(f"No match found for {extracted_file}")


No match found for face_10_sr.jpg
No match found for face_11_sr.jpg
No match found for face_12_sr.jpg
face_13_sr.jpg -> face_22BCE382_0.jpg (Score: 0.76)
face_14_sr.jpg -> face_22BCE382_0.jpg (Score: 0.72)
No match found for face_15_sr.jpg
No match found for face_16_sr.jpg
face_17_sr.jpg -> face_22BCE382_0.jpg (Score: 0.60)
face_18_sr.jpg -> face_22BCE213_0.jpg (Score: 0.71)
face_19_sr.jpg -> face_22BCE213_0.jpg (Score: 0.69)
face_1_sr.jpg -> face_22bce008_0.jpg (Score: 0.79)
face_20_sr.jpg -> face_22BCE269_0.jpg (Score: 0.73)
No match found for face_21_sr.jpg
face_22_sr.jpg -> face_22BCE269_0.jpg (Score: 0.76)
face_23_sr.jpg -> face_22BCE213_0.jpg (Score: 0.75)
face_24_sr.jpg -> face_22BCE382_0.jpg (Score: 0.71)
No match found for face_25_sr.jpg
No match found for face_26_sr.jpg
face_27_sr.jpg -> face_22BCE315_0.jpg (Score: 0.63)
No match found for face_28_sr.jpg
No match found for face_29_sr.jpg
No match found for face_2_sr.jpg
No match found for face_30_sr.jpg
face_31_sr.jpg -> face